In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from torch.optim import lr_scheduler
from torchsummary import summary
import torchvision.datasets
from PIL import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms

In [2]:
device = torch.device('cuda')

In [4]:
trans = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_set = torchvision.datasets.ImageFolder(root ='C:\\Users\\tlseh\\Desktop\\메크로\\HRI_image', transform = trans )
train_loader = DataLoader(dataset = train_set, batch_size = 8, shuffle = True, num_workers=8)

In [5]:
train_data, val_data = torch.utils.data.random_split(train_set,[1000,122])

In [ ]:
train_data.dataset[1120]

(tensor([[[-0.0353, -0.1922, -0.3725,  ..., -0.2627, -0.2471, -0.2941],
          [ 0.0118, -0.0980, -0.3098,  ..., -0.3412, -0.3490, -0.3020],
          [ 0.0745,  0.0039, -0.1765,  ..., -0.6078, -0.4275, -0.3020],
          ...,
          [-0.2000,  0.0039, -0.0353,  ..., -0.0745, -0.1059, -0.4353],
          [-0.2157, -0.0118, -0.0510,  ..., -0.3333, -0.2627, -0.3569],
          [-0.1922,  0.0745, -0.0431,  ..., -0.1922, -0.2314, -0.4824]],
 
         [[ 0.0588, -0.1137, -0.2784,  ..., -0.2314, -0.2784, -0.3098],
          [ 0.1373,  0.0039, -0.2471,  ..., -0.3020, -0.3255, -0.2549],
          [ 0.0745,  0.0588, -0.1373,  ..., -0.5137, -0.3804, -0.2549],
          ...,
          [-0.0902,  0.0196, -0.0824,  ...,  0.0039, -0.0902, -0.3490],
          [-0.1294,  0.0039, -0.1059,  ..., -0.2392, -0.1843, -0.2549],
          [-0.1059,  0.0275, -0.0902,  ..., -0.1216, -0.1529, -0.3490]],
 
         [[ 0.0588, -0.0980, -0.2549,  ..., -0.2706, -0.3176, -0.3333],
          [ 0.2392,  0.0039,

In [6]:

cnt_0 = 0
cnt_1 = 0
cnt_2 = 0
cnt_3 = 0
cnt_4 = 0

for i in range(1000):
    if train_data[i][1] == 0:
        cnt_0+=1
    elif train_data[i][1] == 1:
        cnt_1+=1
    elif train_data[i][1] == 2:
        cnt_2+=1
    elif train_data[i][1] == 3:
        cnt_3+=1
    elif train_data[i][1] == 4:
        cnt_4+=1    
        
#print(sum(cnt_1,cnt_0,cnt_2,cnt_3,cnt_4))
print(cnt_0,cnt_1,cnt_2,cnt_3,cnt_4)

27 238 568 157 10


In [9]:
weights = torch.tensor([cnt_0,cnt_1,cnt_2,cnt_3,cnt_4], dtype= torch.float32)
print(weights.sum())
weights = weights.sum()/weights
print(weights)
weights = weights/weights.sum()
print(weights)
print(weights.sum())
weights = torch.tensor([weights[0],weights[1],weights[2],weights[3],weights[4],0,0,0,0,0], dtype= torch.float32)
print(weights, weights.sum())
                       
class_weights = torch.FloatTensor(weights).cuda()

tensor(1000.)
tensor([ 37.0370,   4.2017,   1.7606,   6.3694, 100.0000])
tensor([0.2480, 0.0281, 0.0118, 0.0426, 0.6695])
tensor(1.0000)
tensor([0.2480, 0.0281, 0.0118, 0.0426, 0.6695, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000]) tensor(1.0000)


In [10]:
class_weights.size()

torch.Size([10])

In [11]:
class LeNet5_model(nn.Module):
    def __init__(self):
        super(LeNet5_model,self).__init__()
        #(3,32,32)
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5,stride = 1, padding= 0)
        #(6,28,28)
        self.pooling1 = nn.MaxPool2d(kernel_size = 2, stride= 2)
        #(6,14,14)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride = 1, padding= 0)
        #(16,10,10)
        self.pooling2 = nn.MaxPool2d(kernel_size = 2, stride= 2)
        #(16,5,5)
        self.fc_out1 = nn.Linear(400,128)
        self.fc_out2 = nn.Linear(128,10)         
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pooling1(x)
        x = F.relu(self.conv2(x))
        x = self.pooling2(x)
        x = x.view(-1,400)
        x = F.relu(self.fc_out1(x))
        x = self.fc_out2(x)
        return x
        
model = LeNet5_model()
print(model)

LeNet5_model(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pooling2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_out1): Linear(in_features=400, out_features=128, bias=True)
  (fc_out2): Linear(in_features=128, out_features=10, bias=True)
)


In [12]:
model.to(device)

LeNet5_model(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pooling2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_out1): Linear(in_features=400, out_features=128, bias=True)
  (fc_out2): Linear(in_features=128, out_features=10, bias=True)
)

In [13]:
batch_size = 128
train_batch_loader = DataLoader(train_data, batch_size, shuffle=True)
val_batch_loader = DataLoader(val_data, batch_size, shuffle=False)

optimizer = optim.Adam(model.parameters(),lr = 0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights) #weight=class_weights
lossfunc = nn.CrossEntropyLoss()

In [14]:
start_time = time.time()
highest_val_acc = 0
loss_list = []
val_acc_list = []
global_steps = 0
epoch = 0
my  = []

print('========================================')
print("Start training...")
while True:
    train_loss = 0
    train_batch_cnt = 0
    model.train()
    for img, label in train_batch_loader:
        global_steps += 1
        # img.shape: [128,3,32,32]
        # label.shape: [128]
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        train_loss += loss
        train_batch_cnt += 1

        #loss_list.append(loss)        

        if global_steps >= 15000:
            print("Training finished.")
            break
        elif epoch >= 100:
            break

    ave_loss = train_loss / train_batch_cnt
    loss_list.append(ave_loss)
    training_time = (time.time() - start_time) / 60
    print('========================================')
    print("epoch:", epoch + 1, "/ global_steps:", global_steps)
    print("training dataset average loss: %.3f" % ave_loss)
    print("training_time: %.2f minutes" % training_time)

    # validation (for early stopping)
    correct_cnt = 0
    model.eval()
    for img, label in val_batch_loader:
        img = img.to(device)
        label = label.to(device)
        pred = model.forward(img)
        _, top_pred = torch.topk(pred, k=1, dim=-1)
        top_pred = top_pred.squeeze(dim=1)
        correct_cnt += int(torch.sum(top_pred == label))
        #print('top_pred:',top_pred)
        #print('label:',label)

    val_acc = correct_cnt / len(val_data) * 100
    #print("validation dataset accuracy: %.2f" % val_acc)
    #print(abs(label - top_pred).sum()/len(label))
    my.append(abs(label - top_pred).sum()/len(label))
    print(my)
    print(correct_cnt,"/",len(val_data),"*********************")
    val_acc_list.append(val_acc)
    if val_acc > highest_val_acc:
        save_path = './' + str(epoch + 1) + '.pth'
        # 위와 같이 저장 위치를 바꾸어 가며 각 setting의 epoch마다의 state를 저장할 것.
        torch.save({'epoch': epoch + 1,
                    'model_state_dict': model.state_dict()},
                    save_path)
        highest_val_acc = val_acc
    epoch += 1
    if global_steps >= 15000:
        break
    elif epoch >= 100:
            break

Start training...
epoch: 1 / global_steps: 8
training dataset average loss: 2.193
training_time: 0.43 minutes
[tensor(1.5984, device='cuda:0')]
9 / 122 *********************
epoch: 2 / global_steps: 16
training dataset average loss: 1.744
training_time: 0.74 minutes
[tensor(1.5984, device='cuda:0'), tensor(1.1803, device='cuda:0')]
23 / 122 *********************
epoch: 3 / global_steps: 24
training dataset average loss: 1.583
training_time: 1.04 minutes
[tensor(1.5984, device='cuda:0'), tensor(1.1803, device='cuda:0'), tensor(1.6311, device='cuda:0')]
9 / 122 *********************
epoch: 4 / global_steps: 32
training dataset average loss: 1.616
training_time: 1.35 minutes
[tensor(1.5984, device='cuda:0'), tensor(1.1803, device='cuda:0'), tensor(1.6311, device='cuda:0'), tensor(0.9016, device='cuda:0')]
32 / 122 *********************
epoch: 5 / global_steps: 40
training dataset average loss: 1.502
training_time: 1.65 minutes
[tensor(1.5984, device='cuda:0'), tensor(1.1803, device='cuda:

In [21]:
model.forward(img)

tensor([[ -9.4723,  13.6556,  16.9934,  ..., -17.6684, -23.2247, -21.4868],
        [ -7.8905,  13.9535,  15.7572,  ..., -17.0728, -20.1476, -16.7468],
        [  8.5237,  12.3259,  12.3592,  ..., -12.8070, -15.8168, -14.0841],
        ...,
        [-11.5063,   9.3580,  11.4894,  ..., -11.8837, -13.5443, -12.5455],
        [ 14.2812,  11.0087,   9.7732,  ..., -11.4113, -14.1104, -13.1227],
        [  9.2038,  15.7316,  15.9506,  ..., -14.6572, -17.4329, -16.1956]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
import math

print(abs(label - top_pred).sum()/len(label))

tensor(1.0492, device='cuda:0')
